In [2]:
import pandas as pd
import numpy as np
import os
import time

In [6]:
def Merges(list1, list2):
    
    a = pd.DataFrame(list1)
    b = pd.DataFrame(list2)
    
    c = pd.merge(a,b, on = a.index)
    c.columns = range(c[c.index==0].size)
    c.set_index(c[0])
    c=c.drop(0,1)
    
    return c

In [7]:


folder='VDS_FAST'
path = r'%s'%folder

path2 = r'VDS_FAST'

df=pd.DataFrame()

file_list= os.listdir(path2)


start = time.time()

# 정상/오류 파일 개수 
count = 0
error_count=0

#진행 상황
m_sum = 0
tmp = -1
m_all = 0

d=len(file_list)
excs = ['.ipynb_checkpoints']
for i in file_list[:d]:
    
    strs = folder+'/'+i
    
    m_ctn = False
    for j in excs: 
        if(i==j):
            m_ctn=True
        else:
            m_ctn=False
    if(m_ctn):
        continue
    
    
    m_stat = os.stat(strs)
    m_size=m_stat.st_size
    m_sum+=m_size

m_all = m_sum
m_sum = 0

print(folder,' : %.2fMB' % (m_all/(1024*1024)))

for i in file_list[:d]:

    strs = folder+'/'+i
    
    m_ctn = False
    
    for j in excs:
        
        if(i==j):
            m_ctn=True
        else:
            m_ctn=False
    
    if(m_ctn):
        
        continue
    
    try:
        df_tmp = pd.read_csv(strs, sep="|", encoding='CP949', error_bad_lines=False, warn_bad_lines=False)
        df=df.append(df_tmp,sort=False)
        del df_tmp
    except:
        print(strs," 파일에서 에러발생")
        error_count+=1
    
    count+=1
        
    m_stat = os.stat(strs)
    m_size=m_stat.st_size
    m_sum+=m_size
    now = int(m_sum/m_all*100)
    if(now>tmp):
        print('%3d%%' % now, end='', flush = True)
    tmp = now

df.to_csv(r'test_fast.csv')

end = time.time()

print('\n소요시간 : %d초 (%d분)' % (int(end-start),(int(end-start)/60)))
print('총 파일 %6d개' % count)
print('총 에러 %6d개' % error_count)

df.columns=['일자','시간','콘존ID','차로유형','통행속도','???']
df=df.drop('???',1)
df.head(5)
#==> 소요시간 : 초
#==> 총 파일 1577개
#==> 총 에러 0개

VDS_FAST  : 12.31MB
100%
소요시간 : 0초 (0분)
총 파일      1개
총 에러      0개


,일자,시간,콘존ID,차로유형,통행속도
0,20170715,805,0500CZS230,1,99.80
1,20170715,805,0500CZS240,1,103.12
2,20170715,805,0500CZS250,1,95.58
3,20170715,805,0500CZS260,1,101.83
4,20170715,805,0500CZS270,1,0.00


In [8]:
#########
#########
#########
#########

df2 = pd.read_csv('test_fast.csv',sep=",")
df2= df2.set_index('Unnamed: 0')

df2.head(5)

,일자,시간,콘존ID,차로유형,통행속도
Unnamed: 0,,,,,
0,20170715,805,0500CZS230,1,99.80
1,20170715,805,0500CZS240,1,103.12
2,20170715,805,0500CZS250,1,95.58
3,20170715,805,0500CZS260,1,101.83
4,20170715,805,0500CZS270,1,0.00


In [223]:
tmp = pd.DataFrame()

In [232]:
cid = '0500CZS230'


tmp_df = df_cz2[df_cz2['콘존ID']==cid]


#print(tmp_df['콘존ID'].notnull().any())
tmp.head(15)

,콘존ID,위도,경도
0,05003CZS272ㅇ0,0.000000,0.000000
0,05003CZS2720,0.000000,0.000000
0,05003CZS2720,0.000000,0.000000
0,05003CZS2720,0.000000,0.000000
5144,0500CZS270,37.977287,128.435622
0,0500CZS2700500CZS230,0.000000,0.000000
5088,0500CZS230,37.691969,127.541312
5088,0500CZS230,37.691969,127.541312
5088,0500CZS230,37.691969,127.541312


In [ ]:
c=pd.DataFrame()
tmp = 0
now = 0
start = time.time()
for cid in df2['콘존ID']:
    
    tmp_df = df_cz2[df_cz2['콘존ID']==cid]
    if tmp_df.empty:
        c=c.append(pd.DataFrame({'콘존ID':[cid],"위도":[0],"경도":[0]}))
    else:
        c=c.append(tmp_df)

    
    now = int(count/len(df2)*100)
    if(now>tmp):
        print('%3d%%' % now, end='', flush = True)
    tmp = now
end = time.time()
tt = int(end-start)
print('')
print('[완료] %d초 소요'%tt)
print('len(c) : %d'%len(c))
print('error : %d'%len(df2)-len(c))
c.head(5)


In [120]:
cols = df2.columns.values
cols = np.append(tmp,c.columns.values)

d = Merges(df2,c)
d.columns=cols

KeyError: RangeIndex(start=0, stop=288863, step=1)

In [126]:
c.index = range(len(c))
len(df2)-len(c)

86448

In [3]:
df_cz = pd.read_csv('acc_conzone.csv',sep=',',encoding='CP949')
df_cz = df_cz.drop('Unnamed: 0',1)
str_arr= ['콘존ID','위도','경도']
df_cz2 = df_cz[str_arr]

print(len(df_cz2))
#df_cz[df_cz['콘존ID']=='0010CZE011']

#중복제거
df_cz2 = df_cz2.drop_duplicates('콘존ID',keep='first')
print(len(df_cz2))

df_cz2.head(5)

6301


,콘존ID,방향,grs80x,grs80y,노선번호,노선명,위도,경도
1,0010CZE011,E,391642.593415,298552.164856,10,경부선,35.265211,129.105973
2,0010CZE011,E,391873.608131,299421.834737,10,경부선,35.273000,129.108714


1030


,콘존ID,위도,경도
0,0010CZE010,35.256977,129.101614
1,0010CZE011,35.265211,129.105973
3,0010CZE020,35.283054,129.104303
4,0010CZE030,35.287171,129.100226
5,0010CZE035,35.293179,129.089296


In [5]:
df.head(5)

NameError: name 'df' is not defined

In [63]:


start =time.time()
for i in range(len(df)):
    
    tmp_df = df_cz2[df_cz2['콘존ID']==cid]

    df[df['콘존ID']==cid]['위도']=tmp_df['위도']
    df[df['콘존ID']==cid]['경도']=tmp_df['경도']
    
    
    now = int(count/len(df)*100)
    if(now>tmp):
        print('%3d%%' % now, end='', flush = True)
    tmp = now
    
end =time.time()
tt = int(end-start)
print('\n%d 초 소요'%tt)

c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


KeyboardInterrupt: 

,일자,시간,콘존ID,차로유형,통행속도,위도,경도
375306,20170715,2355,5510CZE030,1,88.70,0,0
375307,20170715,2355,5510CZS005,1,96.24,0,0
375308,20170715,2355,5510CZS010,1,103.57,0,0
375309,20170715,2355,5510CZS020,1,109.38,0,0
375310,20170715,2355,5510CZS030,1,92.79,0,0


In [4]:
## 통행량

#highway/data/VDS_COUNT
folder='VDS_COUNT'
path = r'%s'%folder

df1=pd.DataFrame(
file_list= os.listdir(path)

start = time.time()

# 정상/오류 파일 개수 
count = 0
error_count=0

#진행 상황
m_sum = 0
tmp = -1
m_all = 0

d=len(file_list)

for i in file_list[:d]:
    
    strs = folder+'/'+i
    m_stat = os.stat(strs)
    m_size=m_stat.st_size
    m_sum+=m_size

m_all = m_sum
m_sum = 0

print(folder,' : %.2fMB' % (m_all/(1024*1024)))

for i in file_list[:d]:

    strs = folder+'/'+i
    
    try:
        df_tmp = pd.read_csv(strs, sep="|", encoding='CP949', error_bad_lines=False, warn_bad_lines=False)
        df1=df1.append(df_tmp,sort=False)
        del df_tmp
    except:
        #print(strs," 파일에서 에러발생")
        error_count+=1
    
    count+=1
        
    m_stat = os.stat(strs)
    m_size=m_stat.st_size
    m_sum+=m_size
    now = int(m_sum/m_all*100)
    if(now>tmp):
        print('%3d%%' % now, end='', flush = True)
    tmp = now
    
end = time.time()

print('\n소요시간 : %d초 (%d분)' % (int(end-start),(int(end-start)/60)))
print('총 파일 %6d개' % count)
print('총 에러 %6d개' % error_count)

df1.columns=['일자','시간','콘존ID','차로유형','통행량','???']
df1=df1.drop('???',1)
df1.head(5)
#==> 소요시간 : 초
#==> 총 파일 1577개
#==> 총 에러 0개

SyntaxError: invalid syntax (<ipython-input-4-0af0a164c8e7>, line 12)